In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils_icpr
from mrcnn import visualize_frame_relation_4f
from mrcnn.visualize_frame_relation_4f import display_images #Raju: change to "visualize"
import mrcnn.model as modellib
from mrcnn.model import log

import custom #Raju: for bridge change it to "custom" 

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/roi20190222T2122/mask_rcnn_roi_0055.h5"  # TODO: update the directory

In [ ]:
config = custom.CustomConfig() #Raju: for bridge change "custom_matc" to "custom"
custom_DIR = os.path.join(ROOT_DIR, "customImages")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
# Here, two parameters we should change
# 1. backbone: resnet 50, resnet101-- from config.py
# 2. Detection_Min_Confidence: 0.5/0.9,... ... --from config.py and custom.py
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Load validation dataset
dataset = custom.CustomDataset() #Raju: for bridge change "custom_matc" to "custom"
dataset.load_custom(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

#### The following line is only the detection code

In [ ]:
from mrcnn.visualize_frame_relation_4f import display_instances_amazon #Raju: change to "visualize"

In [ ]:
# My original detection code:
# from mrcnn import visualize_frame_relation_4f
#-------------------------------------------------------------
# Test results will be saved in the following directory
testresult = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testresult/report_req/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/report_req/'

# Creating a list with the names of all the test images from the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[14:16]

#looping through all images in the test_dir
for i in range(len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    visualize_frame_relation_4f.display_instances_amazon(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
    plt.savefig(testresult+testfiles[i])  

    
print('done')

### The following code is the detection code with Temporal Coherence

In [ ]:
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testresult/New folder/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/New folder/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[1:150]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        a= cl
        Xcor1 = Xco
        Ycor1 = Yco
        plt.savefig(testresult+testfiles[i])
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        b= cl
        Xcor2 = Xco
        Ycor2 = Yco
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
        plt.savefig(testresult+testfiles[i])

    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        plt.savefig(testresult+testfiles[i])

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

# Precision_Recall with temporal coherence

In [ ]:
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/david_overexposed/'

# Images from the following directory will be tested
# test_dir = os.path.join(ROOT_DIR, "CustomImages/test/test_dir/10frames_bridgeC_part1/")
test_dir = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/david_overexposed/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[1:150]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config,
                           i, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils_icpr.compute_precision_recall(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        a= cl
        Xcor1 = Xco
        Ycor1 = Yco
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils_icpr.compute_precision_recall(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        b= cl
        Xcor2 = Xco
        Ycor2 = Yco
        
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils_icpr.compute_precision_recall(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco

        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils_icpr.compute_precision_recall(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco


    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils_icpr.compute_precision_recall(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        

frequency = 3500  # Set Frequency To 2500 Hertz
duration = 100  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

#### Precision- Recall calculation and saving in a csv file 

In [ ]:
import winsound
from mrcnn import utils
# Test results will be saved in the following directory
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testresult/david_overexposed/'

# Images from the following directory will be tested
test_dir = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/david_overexposed/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[3:4]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor=[]
Ycor=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i, name in enumerate(testfiles):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config,
                           i, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    
    filename= os.path.join(test_dir,testfiles[i])
#     image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
#     ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    Xc= converted[1] # X coordinate of all objects will be stored 
    Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    gt_match, pred_match,overlaps =utils.compute_precision_recall_old_version(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'])
#     visualize_frame_relation_4f.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset.class_names, r['scores'], ax=ax,
#                             title="Predictions")
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    print(class_ids, 'for i =', i)
    print(scores, 'for i =', i)
    print(filename)
      

frequency = 3500  # Set Frequency To 2500 Hertz
duration = 100  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
from mrcnn.visualize_frame_relation_4f import plot_overlaps #Raju: change to "visualize"

In [ ]:
from mrcnn import utils
#mAP calculation:
image_ids = os.listdir('C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/overexposed/')
# image_ids = image_ids[58:60]
image_ids
APs =[]

for image_id, name in enumerate(image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection

    results = model.detect([image], verbose=0)
    r = results[0]
    
#     # Compute AP over range 0.5 to 0.95 and print it
#     utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
#                            r['rois'], r['class_ids'], r['scores'], r['masks'],
#                            verbose=1)

# #     visualize.display_differences(
# #         image,
# #         gt_bbox, gt_class_id, gt_mask,
# #         r['rois'], r['class_ids'], r['scores'], r['masks'],
# #         dataset.class_names, ax=get_ax(),
# #         show_box=False, show_mask=False,
# #         iou_threshold=0.5, score_threshold=0.5)
#     # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
#     print("AP : ",AP , "for image id: ", image_id)

# #     print("precisions :", precisions)
# #     print("recalls :", recalls)
# #     print("overlaps :", overlaps)
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))
# print (APs)

In [ ]:
from mrcnn import utils
# Draw precision-recall curve
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize_frame_relation_4f.plot_precision_recall(AP, precisions, recalls)
print(precisions)
print(recalls)

In [ ]:
# Grid of ground truth objects and their predictions
visualize_frame_relation_4f.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, dataset.class_names)

In [ ]:
import winsound
# Test results will be saved in the following directory
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/icpr/CustomImages/test/testresult/icpr_precision/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/icpr/CustomImages/test/testdir/icpr_precision/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[25:27]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor=[]
Ycor=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i, name in enumerate(testfiles):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config,
                           i, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    
    filename= os.path.join(test_dir,testfiles[i])
#     image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
#     ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    Xc= converted[1] # X coordinate of all objects will be stored 
    Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    print(class_ids, 'for i =', i)
    print(scores, 'for i =', i)

    if i%5==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a, c, d, e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
    
    elif i%4==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)
           
        
        gt_match, pred_match,overlaps =utils.compute_precision_recall(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a, c, d, e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
        
    
    elif i%2==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a, c, d, e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        a= cl
        Xcor = Xco
        Ycor = Yco
        
    elif i%3==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a, c, d, e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        c = cl
        Xcor3 = Xco
        Ycor3 = Yco
        
    else:

        gt_match, pred_match,overlaps =utils.compute_precision_recall(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a, c, d, e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

#### updated detection code with 5 frames temporal window

In [ ]:
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/14Feb_bridgeC_672/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/10frames_bridgeC_part1/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
testfiles = testfiles[1:150]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        a= cl
        Xcor1 = Xco
        Ycor1 = Yco
        plt.savefig(testresult+testfiles[i])
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        b= cl
        Xcor2 = Xco
        Ycor2 = Yco
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
        plt.savefig(testresult+testfiles[i])

    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        plt.savefig(testresult+testfiles[i])

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/12Feb_bridgeC/'

# Images from the following directory will be tested
test_dir ='S:/Images/YDXJ0944_done/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
testfiles = testfiles[738:]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor=[]
Ycor=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i in range(len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image

    if i%5==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,c,d,e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        plt.savefig(testresult+testfiles[i])
    
    elif i%4==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)
           
        
        visualize_frame_relation_4f.display_instances_5f_for_annotation(image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,c,d,e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
        plt.savefig(testresult+testfiles[i])
    
    elif i%2==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]        
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(image,file,filename,filesize,  r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,c,d,e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        a= cl
        Xcor = Xco
        Ycor = Yco
        plt.savefig(testresult+testfiles[i])
    elif i%3==0:
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(image,file,filename,filesize,  r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,c,d,e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        c = cl
        Xcor3 = Xco
        Ycor3 = Yco
        plt.savefig(testresult+testfiles[i])
    else:

        visualize_frame_relation_4f.display_instances_5f_for_annotation(image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,c,d,e,Xcor,Ycor,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        plt.savefig(testresult+testfiles[i])

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
#mAP calculation:
image_ids = os.listdir('C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/precision_5_feb/')
# image_ids = image_ids[2:6]
# image_ids
APs =[]

for image_id, name in enumerate(image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    print(r['rois'], ' roi')
    print(r['class_ids'], ' class ids')
    print(r['scores'], ' scores')
    gt_match, pred_match,overlaps =utils.compute_matches(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    
#     utils.precision(gt_mask,r['scores'], r['masks'])
#     precision
# print(precision)
